<a href="https://colab.research.google.com/github/MammadTavakoli/Download-Video/blob/main/Youtube_dl_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Pakages

In [ ]:
!pip install --upgrade  git+https://github.com/yt-dlp/yt-dlp.git
!pip install --upgrade git+https://github.com/byroot/pysrt.git
!pip install youtube-transcript-api
!pip install --upgrade vtt_to_srt3
!pip install colorama

# Import Pakages

# Download Youtube File

In [ ]:
from vtt_to_srt.vtt_to_srt import ConvertDirectories
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from youtube_transcript_api.formatters import SRTFormatter
from google.colab import drive
from colorama import Fore, Style, init
import yt_dlp
import os
import glob
import re
import pysrt
import copy
import re

In [ ]:
init(autoreset=True)  # Auto-reset color after each print

## Mount Drive

In [ ]:
drive.mount('/content/drive/', force_remount=True)

## Add URL To Downloaded Urls List Function

In [ ]:
def url_list_add(url):
    # Define the path where the file is located
    path = "//content//drive//MyDrive//YouTube//"

    # Create the folder for the downloaded videos
    if not os.path.exists(path):
        os.makedirs(path)

    # Create the complete file path by joining the path and file name
    file_path = os.path.join(path, "_urls.txt")

    # Check if the file exists
    if not os.path.isfile(file_path):
        # If the file doesn't exist, create an empty file
        open(file_path, "w").close()

    # Open the file in read mode
    with open(file_path, 'r', encoding="utf-8") as file:
        # Read the contents of the file
        content = file.read()

        # Check if the url already exists in the file
        if url in content:
            print(f'{url} already exists')
            return False

    # Open the file in append mode
    with open(file_path, "a", encoding="utf-8") as file:
        # Write the url to the file
        file.write(url + "\n")
        print(f'{url} added')

    return True

## Create Video Folder Function

In [ ]:
def create_folder(url, base_path):
    try:
        with yt_dlp.YoutubeDL({"ignoreerrors": True, "quiet": True}) as ydl:
            playlist_dict = ydl.extract_info(url, download=False)
            print("\n", playlist_dict["title"], "\n")

            new_folder = playlist_dict["title"] + "_" + playlist_dict["uploader"]
            folder_name = re.sub(r'[-\s/\\]+', '-', new_folder)
            path = os.path.join(base_path, folder_name)

            # Create the folder for the downloaded videos
            if not os.path.exists(path):
                os.makedirs(path)

            # Change the current working directory to the new folder
            os.chdir(path)
            print(os.getcwd())

            # Save the URL in a text file
            url_file_path = os.path.join(path, "_url.txt")
            with open(url_file_path, "w", encoding="utf-8") as f:
                f.write(url)

            print("Folder created successfully.")
    except yt_dlp.DownloadError as e:
        print("Error downloading video info:", str(e))
    except Exception as e:
        print("Error creating folder:", str(e))


## Download Video Function

In [ ]:
def download_srt(video_info_list, subtitleslangs):
    try:
        for video in video_info_list:
          sublangs = copy.deepcopy(subtitleslangs)
          video_id = video.get('id')
          filename = video.get('filename')

          # save translated transcript
          transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

          for lng in transcript_list:
            lng = lng.language_code
            if lng in  sublangs:

              # Get transcript of video
              srt = YouTubeTranscriptApi.get_transcript(video_id, languages=[lng])

              # Convert to SRT format
              formatter = SRTFormatter()
              en_caption_data = formatter.format_transcript(srt)

              # Save the subtitle in the file
              print(f'Downloading {lng} subtitles for: ', filename)
              with open(rf"{filename}.{lng}.srt", "w", encoding="utf-8") as f:
                f.write(en_caption_data)
              sublangs.remove(lng)


          lng = ''
          for transcript in transcript_list:
            lng = transcript.language_code
            if lng != '':
              break

          if lng != '':
            transcript = transcript_list.find_transcript([lng])

            for tr_lang in sublangs:
              translated_transcript = transcript.translate(tr_lang)

              # Convert to SRT format
              formatter = SRTFormatter()
              en_caption_data = formatter.format_transcript(translated_transcript.fetch())

              # Save the subtitle in the file
              print(f'Downloading {translated_transcript} translated subtitles for: ', filename)
              with open(rf"{filename}.{tr_lang}.srt", "w", encoding="utf-8") as f:
                  f.write(en_caption_data)

    except TranscriptsDisabled:
        print('Subtitles are disabled for:', filename)
    except Exception as e:
        print('Error downloading subtitles for:', filename, 'Error:', str(e))

In [ ]:
def download_video(URL,
                   base_path="//content//drive//MyDrive//YouTube//Download_video//",
                   playlist_start=1,
                   skip_download=False,
                   subtitleslangs = ["en", "fa"],
                   max_resolution=1080):

   match = re.search(r'(?<=v=)[^&#]+', URL)
   if match:
      print(match)
      video_id = match.group()
      URL = f"https://www.youtube.com/watch?v={video_id}"

   # Create the folder for the downloaded videos
   create_folder(URL, base_path)

   video_info_list = []
   # Set default output filename format
   youtube_dl_options = {
       "format": f"(bestvideo[height<={max_resolution}]+bestaudio/best)",
       "outtmpl": "%(autonumber)03d_%(title)s.%(ext)s", # Use autonumbering and title as filename
       "autonumber_start": playlist_start, # Start numbering from specified value
       "playliststart": playlist_start, # Download videos starting from specified index
       "restrictfilenames": False, # Allow special characters in filenames
       "nooverwrites": True, # Prevent overwriting existing files
       "writedescription": True, # Download video description
       "writeinfojson": False, # Download video metadata as JSON
       "writeannotations": True, # Download annotations (comments)
       "writethumbnail": False, # Download thumbnails
       "writesubtitles": False, # Download subtitles
       "writeautomaticsub": False,
       "ignoreerrors": True,
   }

   # Try downloading the video while handling errors
   is_downloaded = False
   while not is_downloaded:
      #  try:
           with yt_dlp.YoutubeDL(youtube_dl_options) as ydl:
               ydl.cache.remove() # Clear cached information
               playlist_info = ydl.extract_info(URL, download=False) # Get video metadata
               print('*'*50)
               ydl.prepare_filename(playlist_info) # Prepare filename based on metadata

               if not skip_download:
                  ydl.download([URL]) # Download the video

              #  print(playlist_info, '*-*'*50)
               if "entries" in playlist_info:
                  for i, entry in enumerate(playlist_info["entries"]):
                      # Check if 'entry' is not None before proceeding
                      if entry is not None:
                        video_id = entry.get("id")
                        title = entry.get("title")
                        j = str(i+playlist_start).zfill(3)
                        filename = f"{j}_{title}"
                        print(filename)
                        video_info_list.append({"id": video_id, "filename": filename})
                      else:
                        print(f"Skipping a None entry at index {i}") # Log the skipped entry
               else:
                    video_id = playlist_info.get("id")
                    title = playlist_info.get("title").replace("_", " | ")
                    filename = f"001_{title}"
                    print(filename)
                    video_info_list.append({"id": video_id, "filename": filename})
               print(video_info_list)
               download_srt(video_info_list, subtitleslangs)
               is_downloaded = True
               return True
      #  except yt_dlp.DownloadError as e:
      #      print(Style.BRIGHT + Fore.RED + "Download error:", str(e))
      #  except Exception as e:
      #      print(Style.BRIGHT + Fore.YELLOW  + "Unexpected error:", str(e))
   print(Fore.RED +'_'*50)


## Urls List

In [ ]:
# The URLS list holds the YouTube video or playlist URLs that you want to download.
# Each URL in the list can be either a string (representing a single video URL) or a dictionary (representing a playlist).
# If it's a dictionary, the dictionary key is the URL of the playlist, and the dictionary value is the index of the video in the playlist where downloading should start.
# If the playlist URL is specified as a string, downloading will start from the first video in the playlist.
# This script allows you to download videos from a YouTube playlist to your Google Drive and then to your PC, with the ability to resume the download from the last video if needed.

# Example of the URLS list:
# URLS = [
#     {"https://www.youtube.com/playlist?list=1234": 3},  # Download from the 3rd video of the playlist
#     {"https://www.youtube.com/playlist?list=1234": 1},  # Download from the first video of the playlist (all videos)
#     "https://www.youtube.com/playlist?list=1234",      # Download from the first video of the playlist (all videos)
#     "https://www.youtube.com/watch?v=1234"              # Download a single video
# ]

URLS = []

## Download Video

In [ ]:
base_path = "//content//drive//MyDrive//YouTube//Download_video//"

for URL in URLS:
    if isinstance(URL, dict) and len(URL) == 1:
        url = list(URL.keys())[0]
        playlist_start = list(URL.values())[0]
    elif isinstance(URL, str):
        url = URL
        playlist_start = 1
    else:
        print('There are some errors in URLS')
        break

    if url_list_add(url):
        print(url)
        print('-' * 50, '\n')

        download_video(url, base_path,
                       playlist_start=playlist_start,
                       skip_download=False,
                       subtitleslangs = ['en', 'fa'],
                       max_resolution=1000)

# Convert subtitle to text

## convert vtt to srt format

In [ ]:
def vtt_to_srt(filepath):
    for root, dirs, files in os.walk(filepath):
        recursive = False
        convert_file = ConvertDirectories(root, recursive, "utf-8")
        convert_file.convert()

## Convert srt to text function

In [ ]:
def str_to_subtitle_to_txt(root_path, appendText='*******'):

    # Convert a single .srt file to .txt
    def process_file(pathname):
        try:
            subs = pysrt.open(pathname)
            texts = ''
            for sub in subs:
                texts += sub.text.replace('\n', ' ') + appendText

            # Construct the text file path
            basename_without_ext = os.path.splitext(os.path.basename(pathname))[0]
            dirname = os.path.dirname(pathname)
            text_filepath = os.path.join(dirname, basename_without_ext + '.txt')

            # Write to the text file and clean up the text
            with open(text_filepath, 'w', encoding='utf-8') as text_file:
                texts = texts.replace('.' + appendText, '.\n\n')
                texts = texts.replace('?' + appendText, '?\n\n')
                texts = texts.replace(appendText, ' ')
                text_file.write(texts)

        except Exception as e:
            print(f"Failed to process file: {pathname}")
            print(f"Error: {e}")

    vtt_to_srt(base_path)

    # Walk through root_path and process all .srt files
    for dirpath, dirnames, files in os.walk(root_path):
        for f in files:
            if f.lower().endswith(".srt"):
                process_file(os.path.join(dirpath, f))

## Convert srt to text

In [ ]:
base_path = "//content//drive//MyDrive//YouTube//Download_video//"

str_to_subtitle_to_txt(base_path)